In [4]:
from torchviz import make_dot
import torch 

In [5]:
from model import Net

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
num_inputs = 360
num_hidden = 1000
num_outputs = 5
num_steps = 1
beta = 0.95


LIF_Model = Net(num_hidden, num_inputs, num_outputs, num_steps, beta).to(device)
print(LIF_Model)

Net(
  (fc1): Linear(in_features=360, out_features=1000, bias=True)
  (lif1): Leaky()
  (fc2): Linear(in_features=1000, out_features=5, bias=True)
  (lif2): Leaky()
)


In [7]:
! pip install onnx

  Obtaining dependency information for onnx from https://files.pythonhosted.org/packages/ff/24/0e522fdcadf0e15fc304145a5b6e5d7246d7f2c507fd9bfe6e1fafb2aa95/onnx-1.14.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 5.0 MB/s eta 0:00:0000:0100:01m


In [8]:
torch.onnx.export(LIF_Model, torch.randn(1, 360).to(device), "images/LIF_Model.onnx", verbose=True )

Exported graph: graph(%onnx::Gemm_0 : Float(1, 360, strides=[360, 1], requires_grad=0, device=cpu),
      %fc1.weight : Float(1000, 360, strides=[360, 1], requires_grad=1, device=cpu),
      %fc1.bias : Float(1000, strides=[1], requires_grad=1, device=cpu),
      %lif1.threshold : Float(requires_grad=0, device=cpu),
      %lif1.beta : Float(requires_grad=0, device=cpu),
      %fc2.weight : Float(5, 1000, strides=[1000, 1], requires_grad=1, device=cpu),
      %fc2.bias : Float(5, strides=[1], requires_grad=1, device=cpu),
      %onnx::Mul_63 : Float(1, 1000, strides=[1000, 1], requires_grad=0, device=cpu),
      %onnx::Mul_70 : Float(1, 5, strides=[5, 1], requires_grad=0, device=cpu)):
  %lif2.beta : Float(requires_grad=0, device=cpu) = onnx::Identity(%lif1.beta)
  %lif2.graded_spikes_factor : Float(requires_grad=0, device=cpu) = onnx::Identity(%lif1.threshold)
  %lif2.threshold : Float(requires_grad=0, device=cpu) = onnx::Identity(%lif1.threshold)
  %lif1.graded_spikes_factor : Float(r

In [10]:
model = torch.onnx.load('images/LIF_Model.onnx')
# load('images/LIF_Model.onnx')
dot = make_dot(model(torch.randn(1, 360)), params=dict(model.named_parameters()))
dot.render('images/LIF_Model', format='png')

AttributeError: module 'torch.onnx' has no attribute 'load'

In [11]:
! pip install onnxruntime

  Obtaining dependency information for onnxruntime from https://files.pythonhosted.org/packages/d6/ee/82f181e561980bc59d53bbf14a32629a27c5c4290cfd193898f0d1933a6d/onnxruntime-1.16.1-cp39-cp39-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 646.0 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 512.5 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.2/6.2 MB 3.1 MB/s eta 0:00:0000:0100:01


In [12]:
import onnxruntime

# Load the ONNX model into a `onnxruntime` session.
sess = onnxruntime.InferenceSession("/home/ubuntu/Desktop/Projects/SNN/ecg_snn/images/LIF_Model.onnx")

# Get the names of the outputs of the model.
output_names = sess.get_output_names()

# For each output, create an image with the output data.
for output_name in output_names:
    output = sess.get_output(output_name)
    image = Image.fromarray(output)
    image.save(output_name + ".png")

AttributeError: 'InferenceSession' object has no attribute 'get_output_names'

In [ ]:
from torchview import gra